<a href="https://colab.research.google.com/github/kenzaLB/TER_Classification_Multitache/blob/main/opinion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #les omports utilisés dans ce notebook 
import sys
from numpy import vstack
import pandas as pd
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
from sklearn.feature_extraction.text import TfidfVectorizer
from  sklearn.model_selection import train_test_split
import pickle
import string

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.pipeline import Pipeline


In [ ]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
my_local_drive='/content/gdrive/MyDrive/TER'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd


#**Classification binaire selon l'opinion**

##**Etape1 : Prétraitement du texte** 





> Fonction MyCleanText **(TransformerMixin)**



*   Mettre le texte en minuscule
*   Se débarasser des stopwords
*   Se débarasser des nombres
*   Stemmatisation
*   Lemmatisation


> Fonction TextNormalizer 


*   Fit_transform de mon corpus propre











In [ ]:

#...................................................Fonction MyCleanText .............................................................................
# mettre en minuscule
#enlever les stopwords
#se debarasser des nombres
#stemmatisation
#lemmatisation 
#......................................................................................................................................................


nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

def MyCleanText(X,
                lowercase=False, #mettre en minuscule
                removestopwords=False, #supprimer les stopwords
                removedigit=False, #supprimer les nombres
                getstemmer=False, #conserver la racine des termes
                getlemmatisation=False #lemmatisation des termes
                ):
    sentence=str(X)
    #suppression des caractères spéciaux
    sentence = re.sub(r'[^\w\s]',' ', sentence)
    # suppression de tous les caractères uniques
    sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)
    # substitution des espaces multiples par un seul espace
    sentence = re.sub(r'\s+', ' ', sentence, flags=re.I)

    # decoupage en mots
    tokens = word_tokenize(sentence)
    if lowercase:
          tokens = [token.lower() for token in tokens]

    # suppression ponctuation
    table = str.maketrans('', '', string.punctuation)
    words = [token.translate(table) for token in tokens]

    # suppression des tokens non alphabetique ou numerique
    words = [word for word in words if word.isalnum()]
    
    # suppression des tokens numerique
    if removedigit:
        words = [word for word in words if not word.isdigit()]

    # suppression des stopwords
    if removestopwords:
        words = [word for word in words if not word in stop_words]

    # lemmatisation
    if getlemmatisation:
        lemmatizer=WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word)for word in words]
        

    # racinisation
    if getstemmer:
        ps = PorterStemmer()
        words=[ps.stem(word) for word in words]
        
    sentence= ' '.join(words)
  
    return sentence   

In [ ]:

#..................................................Etape 1 : prétraitement du texte ...................................................................................................
#...................................................Fonction TextNormalizer  .............................................................................
#fit_transform de mon corpus propre 
#......................................................................................................................................................

from sklearn.base import BaseEstimator, TransformerMixin

class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 removestopwords=False, # suppression des stopwords
                 lowercase=False,# passage en minuscule
                 removedigit=False, # supprimer les nombres  
                 getstemmer=False,# racinisation des termes 
                 getlemmatisation=False # lemmatisation des termes  
                ):
        
        self.lowercase=lowercase
        self.getstemmer=getstemmer
        self.removestopwords=removestopwords
        self.getlemmatisation=getlemmatisation
        self.removedigit=removedigit

    def transform(self, X, **transform_params):
        # Nettoyage du texte
        X=X.copy() # pour conserver le fichier d'origine
        return [MyCleanText(text,lowercase=self.lowercase,
                            getstemmer=self.getstemmer,
                            removestopwords=self.removestopwords,
                            getlemmatisation=self.getlemmatisation,
                            removedigit=self.removedigit) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X).transform(X)

    def get_params(self, deep=True):
        return {
            'lowercase':self.lowercase,
            'getstemmer':self.getstemmer,
            'removestopwords':self.removestopwords,
            'getlemmatisation':self.getlemmatisation,
            'removedigit':self.removedigit
        }    
    
    def set_params (self, **parameters):
        for parameter, value in parameters.items():
            setattr(self,parameter,value)
        return self    


##**Etape 2 : Prepare the Data**
* Load and prepare your data 
* Neural network models require numerical input
* Data and numerical output data 


In [ ]:
df = pd.read_csv("ReviewsLabelled.csv", names=['sentence','sentiment','source'], header=0,sep='\t', encoding='utf8')
print("Echantillon de mon dataset \n")
print(df.head())
print("\n")
print("Quelques informations importantes \n")
df.info()


**Dataset Class :**

* Constructor of the dataset object loads the data file .

* the __len__() function can be used to get the number of rows of the dataset 

* the __get_item__() function is udes to get a specific sample by index

* **The random_split** function can be used to split a dataset into train and test sets .Once split ,a selection of rows from the Dataset can be provided to a DataLoader , along with the batch size and wether the data should be shuffled every epoch .




In [ ]:
#....................................... CSVDataset .............................................
# j'obtiens une matrice avec de 0
# utilisation TextNormalizer et Tfidf 
# c'est elle qui me parait la plus pertinente pour l'instant 
#...................................................................................................
NB_FEATURES=1000
EPOCHS=150


class CSVDataset(Dataset):
        # load the dataset
        def __init__(self, path):
            # load the csv file as a dataframe
            df = read_csv(path,header=0, sep='\t', encoding='utf8')
            print(df.head())
            # selection des données (inputs and outputs)
            self.X = df.values[:, 0:1]
            self.y = df.values[:, -2]
            #création du pipeline 
            pipe = Pipeline([("cleaner", TextNormalizer(lowercase=True)),
                 ("tfidf_vectorizer", TfidfVectorizer(max_features=NB_FEATURES))])
            self.X=pipe.fit_transform(self.X)
            # sauvegarde du vocabulaire du TF_IDF pour tester d'autres données
            pickle.dump(pipe.named_steps['tfidf_vectorizer'].vocabulary_,open("vocabulaire.txt","wb"))
            # attention important il faut convertir le résultat de TF_IDF en array et dire que dedans ce 
            # sont des floats
            self.X=self.X.toarray()
            self.X = self.X.astype('float32')
            # ne pas oublier de transformer le y via label encoder -> chaque classe aura un numéro (meme si je ne pense pas que ce soit utile dans le cas de l opinion)
            #self.y = LabelEncoder().fit_transform(self.y)
            ########################################### ANCIENNE VERSION ################################################################
            #text_normalizer=TextNormalizer(lowercase=True,getstemmer=True,removestopwords=True,getlemmatisation=True,removedigit=True) 
            #appliquer fit.transform pour réaliser les pretraitements sur X 
            #self.X=text_normalizer.fit_transform(self.X)
            #tfidf=TfidfVectorizer(ngram_range=(2, 2))
            #self.X=tfidf.fit_transform(self.X).toarray()
            #print("Mon nombre de colonnes est : ",self.X.shape[1])
            #self.X=self.X.astype("float32")
            
           
            

        # number of rows in the dataset
        def __len__(self):
             #return self.X.shape[1]
             return len(self.X)

        # get a row at an index
        def __getitem__(self, idx):
            return [self.X[idx], self.y[idx]]

        # get indexes for train and test rows
        def get_splits(self, n_test=0.30):
        # determine sizes
            test_size = round(n_test * len(self.X))
            print("test_size",test_size)
            train_size = len(self.X) - test_size
            print("train_size",train_size)
            # calculate the split
            return random_split(self, [train_size, test_size])




In [ ]:
ls

In [ ]:
path="ReviewsLabelled.csv"
print("création de mon objet Dataset \n")
dataset=CSVDataset(path)

#probleme les modifications ne sont pas prises en compte ,mon texte comprte encore des majuscules etc ..
print("Le set Obtenu \n")
print(dataset.X)

print("Le nombre de rows de mon Dataset :\n")
print(dataset.__len__())

print("le type de mon X \n")
type(dataset.X)

print(len(dataset.X))
print("L'item à l'indice 100 \n")
print(dataset.__getitem__(100))

##**NB:**
A DataLoader instance can be created for the training dataset ,testdataset and even the a validation dataset .


##**Etape 3 : Define the Model**
To define a model in Pytorch we need to define a class that extends the Module Class

* **The constructor :** defines the layers of the model.

* **Forward Function :** defines how to forward propagate input through the defined layers of the model.

**NB:** Many layers are available such as Linear for fully connected layers , Conv2d etc ..

--> We are gonna use Linear 


In [ ]:
class MLP(Module):
        # define model elements
        def __init__(self,n_inputs ):
            super(MLP, self).__init__()
            # input to first hidden layer
            self.hidden1 = Linear(n_inputs, 25) #13465
            kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
            self.act1 = ReLU()
            # second hidden layer
            self.hidden2 = Linear(25, 20)
            kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
            self.act2 = ReLU()
            # third hidden layer and output
            self.hidden3 = Linear(20, 1)
            xavier_uniform_(self.hidden3.weight)
            self.act3 = Sigmoid()

        # forward propagate input
        def forward(self, X):
            # input to first hidden layer
            X = self.hidden1(X)
            X = self.act1(X)
            # second hidden layer
            X = self.hidden2(X)
            X = self.act2(X)
            # third hidden layer and output
            X = self.hidden3(X)
            X = self.act3(X)
            return X


In [ ]:
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    print(len(dataset.X))
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True) #remet shuffle a True
    #shuffle=False veut dire que je decide de pas mélanger 
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)

    return train_dl, test_dl


##**Etape 5 : Train the Model**
The training process requires to define a loss function and an optimization algorithm .

* **BCELoss** : for binary classification 

* CrossEntropyLoss : for multi-class classification  






A loop is required for the number of training epochs 


```
for epoch in range (100)
```

Then an inner loop is required for the mini batches for the gradient descent



```
for i,(inputs,targets) in enumerate(train_dl)
```
Each update to the model involves the same general pattern comprised of:

* Clearing the last error gradient.

* A forward pass of the input through the model.

* Calculating the loss for the model output.

* Backpropagating the error through the model.
* Update the model in an effort to reduce loss.


In [ ]:
from sklearn.utils import shuffle
from torch.nn.modules.loss import BCELoss
def train_model(train_dl, model):
            # define the optimization
            criterion = BCELoss() #fonction cout 
            optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9) #gradiant descent (momentum c'est une simple optimisation pour améliorer le learning rate )
            # enumerate epochs
            for epoch in range(500):
                # enumerate mini batches
                for i, (inputs, targets) in enumerate(train_dl):
                #for data in DataLoader(train ,batch_size = 32 ,shuffle=True)
                    targets=targets.unsqueeze(-1) #je l'ai mis car j'avais des erreurs de shape entre targets et inputs 
                    # clear the gradients
                    optimizer.zero_grad()              
                    # compute the model output
                    yhat = model(inputs.float())
                    # calculate loss
                    loss = criterion(yhat, targets.float())
                    # credit assignment
                    loss.backward()
                    # update model weights
                    optimizer.step()


##**Etape 6 : Evaluate the Model**

In [ ]:
# evaluate the model
def evaluate_model(test_dl, model):
                        predictions, actuals = list(), list()
                        for i, (inputs, targets) in enumerate(test_dl):
                            # evaluate the model on the test set
                            yhat = model(inputs)
                            # retrieve numpy array
                            yhat = yhat.detach().numpy()
                            actual = targets.numpy()
                            actual = actual.reshape((len(actual), 1))
                            # round to class values
                            yhat = yhat.round()
                            # store
                            predictions.append(yhat)
                            actuals.append(actual)
                            predictions, actuals = vstack(predictions), vstack(actuals)
                            #print(predictions) -> je voulais juste voir si c'étaient reellement des 0 et des 1 -> c'est le cas
                            # calculate accuracy
                            acc = accuracy_score(actuals, predictions)
                            return acc


##**Etape 7: Make predictions**

In [ ]:
from nltk.sem.logic import Variable

# make a class prediction for one row of data
def predict(row, model):
                # convert row to data
                #row = Variable(Tensor([row]).float())
                row=Tensor([row])
                # make prediction
                yhat = model(row)
                # retrieve numpy array
                yhat = yhat.detach().numpy()
                return yhat


In [ ]:

# prepare data
path = 'ReviewsLabelled.csv'
train_dl, test_dl = prepare_data(path)
print("Len train_dl = nb batchsize", len(train_dl), len(train_dl.dataset), len(test_dl.dataset))




In [ ]:

#print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
nbrows=dataset.X.shape[1]
model = MLP(nbrows)




In [ ]:
# train the model
print(len(train_dl.dataset), len(test_dl.dataset))
train_model(train_dl, model)

In [ ]:
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)


In [ ]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

***TESTS DE PREDICT***

In [ ]:
# Creation du modele general sur toutes les donnée pour faire de la prédiction

# NORMALEMENT ICI IL FAUT RELANCER L'APPRENTISSAGE SUR TOUT LE JEU DE DONNEES POUR AVOIR UN BON MODELE


# EN DESSOUS C'EST POUR MONTRER COMMENT ON PEUT UTILISER LE MODELE. JE N'AI PAS TROP TESTE. 
# J'AI JUSTE VERIFIE QUE LES PRETRAIREMENTS ETAIENT BIEN FAIT : on nettoie et on fait le tf-idf avec le vocabulaire
# sur lequel on a appris
clean_data=pickle.load( open( "vocabulaire.txt", "rb" ) )

# make a single prediction
row = ["The mic is great."]
#, "The SKY IS BLUE", 'my first visit to Hiro was a delight !'] # Amazon
row=TextNormalizer(lowercase=True).fit_transform(row)
#row=clean_data.named_steps["cleaner"].fit_transform(row)
print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)
row=vectorizer2.fit_transform(row).toarray()
yhat = predict(row, model)
print('Predicted: %d' % round(yhat[0][0][0]))



In [ ]:


row = ["Instead, we got a bore fest about a whiny, spoiled brat babysitting."] #imdb
row=TextNormalizer(lowercase=True).fit_transform(row)
#row=clean_data.named_steps["cleaner"].fit_transform(row)
print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
yhat = predict(row, model)
#print(round(yhat[0][0][0]))
print('Predicted: %d' % round(yhat[0][0][0]))

#row = [My first visit to Hiro was a delight!] yeld

In [ ]:

row = ["Good case, Excellent value"] #imdb
row=TextNormalizer(lowercase=True).fit_transform(row)
#row=clean_data.named_steps["cleaner"].fit_transform(row)
print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
yhat = predict(row, model)
print('Predicted: %d' % round(yhat[0][0][0]))



#**Classification multiclass selon la source**



In [ ]:
# pytorch mlp for multiclass classification
from numpy import vstack
from numpy import argmax
from pandas import read_csv
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import sys
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
from torchvision import transforms
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from torch.nn import BCELoss
import numpy as np
import seaborn as sns
import tensorflow as tf
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk import word_tokenize
import torch
from torch import nn
import scipy
from torch import optim
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

def MyCleanText(X, 
               lowercase=False, # mettre en minuscule
               removestopwords=False, # supprimer les stopwords
               removedigit=False, # supprimer les nombres  
               getstemmer=False, # conserver la racine des termes
               getlemmatisation=False # lematisation des termes 
              ):
    
    sentence=str(X)

    # suppression des caractères spéciaux
    sentence = re.sub(r'[^\w\s]',' ', sentence)
    # suppression de tous les caractères uniques
    sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)
    # substitution des espaces multiples par un seul espace
    sentence = re.sub(r'\s+', ' ', sentence, flags=re.I)

    # decoupage en mots
    tokens = word_tokenize(sentence)
    if lowercase:
          tokens = [token.lower() for token in tokens]

    # suppression ponctuation
    table = str.maketrans('', '', string.punctuation)
    words = [token.translate(table) for token in tokens]

    # suppression des tokens non alphabetique ou numerique
    words = [word for word in words if word.isalnum()]
    
    # suppression des tokens numerique
    if removedigit:
        words = [word for word in words if not word.isdigit()]

    # suppression des stopwords
    if removestopwords:
        words = [word for word in words if not word in stop_words]

    # lemmatisation
    if getlemmatisation:
        lemmatizer=WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word)for word in words]
        

    # racinisation
    if getstemmer:
        ps = PorterStemmer()
        words=[ps.stem(word) for word in words]
        
    sentence= ' '.join(words)
  
    return sentence   

from sklearn.base import BaseEstimator, TransformerMixin

class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 removestopwords=False, # suppression des stopwords
                 lowercase=False,# passage en minuscule
                 removedigit=False, # supprimer les nombres  
                 getstemmer=False,# racinisation des termes 
                 getlemmatisation=False # lemmatisation des termes  
                ):
        
        self.lowercase=lowercase
        self.getstemmer=getstemmer
        self.removestopwords=removestopwords
        self.getlemmatisation=getlemmatisation
        self.removedigit=removedigit
        #self.transform = transforms.Compose([transforms.ToTensor()])  

    def transform(self, X, **transform_params):
        # Nettoyage du texte
        X=X.copy() # pour conserver le fichier d'origine
        return [MyCleanText(text,lowercase=self.lowercase,
                            getstemmer=self.getstemmer,
                            removestopwords=self.removestopwords,
                            getlemmatisation=self.getlemmatisation,
                            removedigit=self.removedigit) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X).transform(X)

    def get_params(self, deep=True):
        return {
            'lowercase':self.lowercase,
            'getstemmer':self.getstemmer,
            'removestopwords':self.removestopwords,
            'getlemmatisation':self.getlemmatisation,
            'removedigit':self.removedigit
        }    
    
    def set_params (self, **parameters):
        for parameter, value in parameters.items():
            setattr(self,parameter,value)
        return self    




In [ ]:
B_FEATURES=1000
EPOCHS=150


# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None, sep='\t', encoding='utf8')
        #print(df.head())
        # store the inputs and outputs
        self.X = df.values[:, :-1]

        self.y = df.values[:, -1]

        # création du pipeline
        pipe = Pipeline([("cleaner", TextNormalizer(lowercase=True)),
                 ("tfidf_vectorizer", TfidfVectorizer(max_features=NB_FEATURES))])


        self.X=pipe.fit_transform(self.X)
        # sauvegarde du vocabulaire du TF_IDF pour tester d'autres données
        pickle.dump(pipe.named_steps['tfidf_vectorizer'].vocabulary_,open("vocabulaire.txt","wb"))

        # attention important il faut convertir le résultat de TF_IDF en array et dire que dedans ce 
        # sont des floats
        self.X=self.X.toarray()
        self.X = self.X.astype('float32')
        # ne pas oublier de transformer le y via label encoder -> chaque classe aura un numéro
        self.y = LabelEncoder().fit_transform(self.y)


    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
       # return self.transform(self.x_data[idx]), self.transform(self.y_data[idx])
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        #self.hidden3 = Linear(8, 3)
        self.hidden3 = Linear(8, 4)
        xavier_uniform_(self.hidden3.weight)
        #self.act3 = Softmax(dim=1)
        self.act3 = Softmax(dim=1)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # output layer
        X = self.hidden3(X)
        return torch.softmax(X, dim=1)
        X = self.act3(X)

        return X

# prepare the dataset
def prepare_data(path):
    # load the dataset
    print("----- dans prepare data avant CSV Dataset")
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    print ("Taille du jeu de test et d'apprentissage",len(train),len(test))
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

# train the model
def train_model(train_dl, model):
    # define the optimization
    #criterion = BCELoss()
    # attention ici c'est une cross entropy
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(EPOCHS):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            
            # clear the gradients
            optimizer.zero_grad()
       
            # compute the model output
            yhat = model(inputs)#.numpy()
            
            # calculate loss
            loss = criterion(yhat, targets)
            
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # convert to class labels
        yhat = argmax(yhat, axis=1)
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat


# prepare the data
path = 'ReviewsLabelled.csv'
train_dl, test_dl = prepare_data(path)

print("Len train_dl = nb batchsize", len(train_dl), len(train_dl.dataset), len(test_dl.dataset))

# define the network
model = MLP(NB_FEATURES)
# train the model
print(len(train_dl.dataset), len(test_dl.dataset))
#train_dl.describe(include = [object])
train_model(train_dl, model)

# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)



#**Attention :le predict de la source est à régler !** 

In [ ]:
clean_data=pickle.load( open( "vocabulaire.txt", "rb" ) )

# make a single prediction
#row = ["The mic is great."]
row=["The SKY IS BLUE, 'my first visit to Hiro was a delight !"]
#, "The SKY IS BLUE", 'my first visit to Hiro was a delight !'] # Amazon
row=TextNormalizer(lowercase=True).fit_transform(row)
#row=clean_data.named_steps["cleaner"].fit_transform(row)
print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
yhat = predict(row, model)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))


row = ["Instead, we got a bore fest about a whiny, spoiled brat babysitting."] #imdb
row=TextNormalizer(lowercase=True).fit_transform(row)
#row=clean_data.named_steps["cleaner"].fit_transform(row)
print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
yhat = predict(row, model)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

#row = [My first visit to Hiro was a delight!] yeld


#**MULTI LABEL CLASSIFICATION**


In [ ]:
#feature -> (une seul ,c'est le review )
#labels -> on en a deux
    #Opinion :binary 
    #Source :Multi Class


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

path ='ReviewsLabelled.csv'
df=read_csv(path,header=0, sep='\t', encoding='utf8')
#print((df.head()))
#print(df[:3000])
print("The initial shape of my DataSet :", df.shape ,"\n")
le=LabelEncoder()
df.source=le.fit_transform(df.source)
#print("\n")
#print("Apres l'application du LABEL ENCODER Sur la SOURCE \n")
#print(df[:3000])
#amazon = 0 , imdb=1 et yelp=2
#on veut d'abord faire que de la classification binaire 
#donc on va se passer de yelp pour l'instant 

df2=pd.DataFrame(df, columns =
                 ['sentence','sentiment' ,'source' ])
indexNames = df2[df2['source']== 2 ].index
df2.drop(indexNames,inplace=True)

print("The shape of My Dataset now that i kept only amazon and imdb dans source :",df2.shape, "\n")
#print(df2[:3000])

df2['target_list'] = df2[['sentiment', 'source']].values.tolist()

final_df=df2[['sentence','target_list']].copy()
print(final_df.head())
#print(df2.head())

labels=final_df.target_list
print(labels)
type(labels[0])


In [ ]:

NB_FEATURES=1000
EPOCHS=150


class InitDataset(Dataset):
        # load the dataset
        def __init__(self, dataframe):
            self.data=dataframe
            self.feature=dataframe['sentence']
            self.labels=self.data.target_list 

            #création du pipeline 
            pipe = Pipeline([("cleaner", TextNormalizer(lowercase=True)),
                 ("tfidf_vectorizer", TfidfVectorizer(max_features=NB_FEATURES))])
            self.feature=pipe.fit_transform(self.feature)
            # sauvegarde du vocabulaire du TF_IDF pour tester d'autres données
            pickle.dump(pipe.named_steps['tfidf_vectorizer'].vocabulary_,open("MultiLabelvocabulaire.txt","wb"))
            # attention important il faut convertir le résultat de TF_IDF en array et dire que dedans ce 
            # sont des floats
            self.feature=self.feature.toarray()
            self.feature = self.feature.astype('float32')
             

        # number of rows in the dataset
        def __len__(self):
             #return self.X.shape[1]
             return len(self.feature)

        # get a row at an index
        def __getitem__(self, idx):
            features=self.feature[idx,:]
            labels=torch.tensor(self.labels[idx], dtype=torch.float32)
            features=torch.tensor(features,dtype=torch.float32)
            label1=torch.tensor(labels[0],dtype=torch.float32)
            label2=torch.tensor(labels[1],dtype=torch.float32)

            return {
                'features ':features,
                'label1':label1,
                'label2':label2,
            }

        # get indexes for train and test rows
        def get_splits(self, n_test=0.30):
        # determine sizes
            test_size = round(n_test * len(self.feature))
            print("test_size",test_size)
            train_size = len(self.X) - test_size
            print("train_size",train_size)
            # calculate the split
            return random_split(self, [train_size, test_size])




#essais
dataset=InitDataset(final_df)
#print(dataset)
dataset.__getitem__(4)


In [ ]:
features=df.sentence
print(X)
y=df.values[:,-2:]
print(y[0])
print("y",y)
Binarylabel=df.values[:,-2]
print(Binarylabel)
MultiLabel=df.values[:,-1]
print(MultiLabel)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class MultiHeadBinaryModel(nn.Module):
    def __init__(self):
        super(MultiHeadBinaryModel, self).__init__()
        self.fc1 = nn.Linear(1, 32) # 1 is the number of features
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, 256)
        
        # we will treat each head as a binary classifier ...
        # ... so the output features will be 1
        self.out1 = nn.Linear(256, 1)
        self.out2 = nn.Linear(256, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        
        # each binary classifier head will have its own output
        out1 = F.sigmoid(self.out1(x))
        out2 = F.sigmoid(self.out2(x))
        
        return out1, out2

In [ ]:
#loss function for binary classification 
import torch.nn as nn

# custom loss function for multi-head binary classification
def binary_loss_fn(outputs, targets):
    o1, o2= outputs
    t1, t2= targets
    l1 = nn.BCELoss()(o1, t1)
    l2 = nn.BCELoss()(o2, t2)
    return (l1 + l2 ) / 2

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

In [ ]:
train_size = 0.8
train_dataset = final_df.sample(frac=train_size,random_state=200)
valid_dataset = final_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(final_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(valid_dataset.shape))

training_set = InitDataset(train_dataset)
validation_set = InitDataset(valid_dataset)

#training data loader
train_dataloader=DataLoader(training_set, shuffle=True, batch_size=1024)
#initialize the model
model=MultiHeadBinaryModel()

In [ ]:
# training function
def train(model, dataloader, optimizer, loss_fn, train_dataset, device):
    model.train()
    counter = 0
    train_running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_dataset)/dataloader.batch_size)):
        counter += 1
        
        # extract the features and labels
        features = data['sentence'].to(device)
        target1 = data['sentiment'].to(device)
        target2 = data['source'].to(device)
        
        # zero-out the optimizer gradients
        optimizer.zero_grad()
        
        outputs = model(features)
        targets = (target1, target2)
        loss = loss_fn(outputs, targets)
        train_running_loss += loss.item()
        
        # backpropagation
        loss.backward()
        # update optimizer parameters
        optimizer.step()
        
    train_loss = train_running_loss / counter
    return train_loss

############################################################################

                                ESSAIS DIVERS 
                        
############################################################################

In [ ]:
#je prepare The row que je vais lui passer
text = ["Good case, Excellent value"]
text_normalizer=TextNormalizer(lowercase=True,getstemmer=True,removestopwords=True,getlemmatisation=True,removedigit=True) 
text=text_normalizer.fit_transform(text)
tfidf=TfidfVectorizer(ngram_range=(2, 2))
text=tfidf.fit_transform(text).toarray()
print(text)

In [ ]:
from sklearn.pipeline import Pipeline
model_pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('model', model())]) #call the Model which you want to use

model_pipeline.fit_transform(x,y) # here x is your text data, and y is going to be your target
model_pipeline.predict(['Food was good"'])  # predict your new sentence

In [ ]:
new = ["I like the ambiance of this hotel "]

pd.DataFrame(tfidf_transformer.transform(count_vect.transform(new)).todense(), 
             columns = count_vect.get_feature_names())


In [ ]:

# make a single prediction (expect class=1)
#c'est la ou je bloque , nb_inputs je l'ai mis en brut dans la definition de mon modele à 13..  , je dois trouver un moyen pour genéraliser 
#et peut être il faut changer la fonction predictit car la elle est supposé recevoir un commentaire , donc elle doit pouvoir faire les prétraitements toute seule 

#row =[0.57735027, 0.57735027, 0.57735027]
#yhat=model.predict(text)

yhat = predict(text, model)
print('Predicted: (class=%d)' % (yhat.round()))




In [ ]:
#...................................................................................................
#............................................PREMIERS ESSAIS      ..................................
#...................................................................................................

import nltk
nltk.download('stopwords') 
X=df.values[:,0:1]

for phrase in X[:10,:] :
  print(phrase)
  phrase[0]= re.sub("[^a-zA-Z]", " ", phrase[0])
  phrase[0]=phrase[0].lower()
  phrase[0] = phrase[0].split()  # diviser enn entités
  ps = PorterStemmer()
  phrase[0]= [ps.stem(word) for word in phrase[0] if not word in set(stopwords.words("english"))]  # tokenization et se debarasser des stopwords
  phrase[0] = ' '.join(phrase[0])
  print(phrase)
  print("....................................\n")

print(X)

In [ ]:
#...............................Troisieme VERSION DE CSVDataset ....................................
#j'obtiens une matrice avec de 0
#utilisation TextNormalizer et Tfidf 
#...................................................................................................

class CSVDataset(Dataset):
        # load the dataset
        def __init__(self, path):
            # load the csv file as a dataframe
            df = read_csv(path,header=0, sep='\t', encoding='utf8')
            # selection des données
            self.X = df.values[:, 0:1]
            self.y = df.values[:, -2]
            text_normalizer=TextNormalizer(lowercase=True,getstemmer=True,removestopwords=True,getlemmatisation=True,removedigit=True)  
            # d'appliquer fit.transform pour appliquer les pré-traitements
            self.X=text_normalizer.fit_transform(self.X)
            print("X apres text_normalizer \n")
            print(self.X)
            # pour l'enchainer avec un tf-idf : 
            tfidf=TfidfVectorizer(ngram_range=(2, 2))
            self.X=tfidf.fit_transform(self.X)
            print("X apres tfidf  \n")
            print(self.X)
            # le vecteur peut par la suite être transformé en matrice : 
            self.X=self.X.toarray()
            

        # number of rows in the dataset
        def __len__(self):
            return len(self.X)

        # get a row at an index
        def __getitem__(self, idx):
            return [self.X[idx], self.y[idx]]

        # get indexes for train and test rows
        def get_splits(self, n_test=0.33):
        # determine sizes
            test_size = round(n_test * len(self.X))
            train_size = len(self.X) - test_size
            # calculate the split
            return random_split(self, [train_size, test_size])

path="ReviewsLabelled.csv"
dataset=CSVDataset(path)
#probleme les modifications ne sont pas prises en compte ,mon texte comprte encore des majuscules etc ..
print("Notre set final \n")
print(dataset.X)
print("La longueur de notre set final est :\n")
print(dataset.__len__())


In [ ]:
#...................................................................................................
#...............................PREMIERE VERSION DE CSVDataset ....................................
#j'obtiens une matrice avec de 0 (est ce que c'est normal ? )
#utilisation de pipeline pour le prétraitement (classe TextNormaliser et Tfidf)
#
#...................................................................................................
from sklearn.pipeline import Pipeline


class CSVDataset(Dataset):
        # load the dataset
        def __init__(self, path):
            # load the csv file as a dataframe
            df = read_csv(path,header=0, sep='\t', encoding='utf8')
            # selection des données
            self.X = df.values[:, 0:1]
            self.y = df.values[:, -2]
            # création du pipeline
            pipe = Pipeline([("cleaner", TextNormalizer()),
                            ("count_vectorizer", TfidfVectorizer())])
            pipe.fit(self.X)
            pipe.transform(self.X)
            # creation du dataframe pour affichage
            df_pipe = pd.DataFrame(
                data=pipe.transform(self.X).toarray(),
                columns=pipe['count_vectorizer'].get_feature_names()
            )
            self.X=df_pipe

            

        # number of rows in the dataset
        def __len__(self):
            return len(self.X)

        # get a row at an index
        def __getitem__(self, idx):
            return [self.X[idx], self.y[idx]]

        # get indexes for train and test rows
        def get_splits(self, n_test=0.33):
        # determine sizes
            test_size = round(n_test * len(self.X))
            train_size = len(self.X) - test_size
            # calculate the split
            return random_split(self, [train_size, test_size])



path="ReviewsLabelled.csv"
dataset=CSVDataset(path)
#probleme les modifications ne sont pas prises en compte ,mon texte comprte encore des majuscules etc ..
print("Notre set final \n")
print(dataset.X)
print("La longueur de notre set final est :\n")
print(dataset.__len__())

print("le type de mon X \n")
dataset.X

In [ ]:
# model definition

class MLP(Module):
        # define model elements
        def __init__(self,n_inputs ):
            super(MLP, self).__init__()
            # input to first hidden layer
            self.hidden1 = Linear(13465, 10)
            kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
            self.act1 = ReLU()
            # second hidden layer
            self.hidden2 = Linear(10, 8)
            kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
            self.act2 = ReLU()
            # third hidden layer and output
            self.hidden3 = Linear(8, 1)
            xavier_uniform_(self.hidden3.weight)
            self.act3 = Sigmoid()

        # forward propagate input
        def forward(self, X):
            # input to first hidden layer
            X = self.hidden1(X)
            X = self.act1(X)
            # second hidden layer
            X = self.hidden2(X)
            X = self.act2(X)
            # third hidden layer and output
            X = self.hidden3(X)
            X = self.act3(X)
            return X




# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        self.layer = Linear(13465, 1)
        self.activation = Sigmoid()

    # forward propagate input
    def forward(self, X):
        X = self.layer(X)
        X = self.activation(X)
        return X


In [ ]:
#exemple d'utilisation 

#print ('Utilisation de MyCleanText avec convertion en minuscule, en prenant les racines, en supprimant les nombres')
#print (MyCleanText(texte,lowercase=True,getstemmer=True, removedigit=True),'\n')

#print ('Utilisation de MyCleanText avec convertion en minuscule et en mettant sous la forme de lemmes')
#print (MyCleanText(texte,lowercase=True,getlemmatisation=True),'\n')

In [ ]:
#...................................................................................................
#...............................TEST DE LA CLASSE TEXTNORMALIZER....................................
#...................................................................................................

texte2 = ["This is an example of TfidfVectorizer for creating a vector",
        "This is another example of TfidfVectorizer",
        "but before we apply a preprocessing"]

print ("texte avant ",texte2)

# il suffit de créer un objet de la classe TextNormalizer

text_normalizer=TextNormalizer(lowercase=True)  
 
# d'appliquer fit.transform pour appliquer les pré-traitements

text_cleaned=text_normalizer.fit_transform(texte2)

print ("texte après application des pré-traitements")

print (text_cleaned)     

# pour l'enchainer avec un tf-idf : 
tfidf=TfidfVectorizer(ngram_range=(2, 2))
vector_tfidf=tfidf.fit_transform(text_cleaned)
print ("texte transformé en vecteur tf-idf")
print (vector_tfidf)

# le vecteur peut par la suite être transformé en matrice : 
print ("transformation du vecteur en matrice")
vector_tfidf.toarray()
print("dimension : \n")

# notons que cette matrice pourra être à l'entrée d'un classifier